In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb && make pipeline`
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_82740/964790820.py:11: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107166,171038,https://blog.aiimpacts.org/p/are-there-example...,Are There Examples of Overhang for Other Techn...,\N,\N,2023-12-14 12:12:11.377056+00,2023-12-14 12:12:11.377056+00,NaN,\N,AI Impacts,17.0,0.0,\N
107167,171039,https://www.askamanager.org/2023/12/update-i-d...,update: I dated someone who was using me to ge...,\N,\N,2023-12-14 12:12:11.37821+00,2023-12-14 12:12:11.37821+00,NaN,\N,AskManager,83.0,0.0,\N
107168,171040,https://www.askamanager.org/2023/12/update-whe...,update: when my boss wants me to do something ...,\N,\N,2023-12-14 12:12:11.379316+00,2023-12-14 12:12:11.379316+00,NaN,\N,AskManager,1.0,0.0,\N
107169,171041,https://thedispatch.com/newsletter/morning/to-...,To Pay or Not To Pay … College Athletes,\N,\N,2023-12-14 12:12:11.380425+00,2023-12-14 12:12:11.380425+00,NaN,\N,Dispatch,17.0,0.0,\N


In [2]:
links['added'].max()

'2023-12-14 12:12:11.38154+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
107118    170990
107119    170991
107120    170992
107121    170993
107122    170994
Name: id, Length: 14790, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred


In [5]:
links[links['url'].duplicated()]['id']

Series([], Name: id, dtype: int64)

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/70
...11/70
...21/70
...31/70
...41/70
...51/70
...61/70
...70/70


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107236,171108,https://twitter.com/aidan_mclau/status/1734621...,"(1) Aidan McLau on X: ""The irony of Mistral re...",\N,\N,2023-12-14 17:07:44.524596,2023-12-14 17:07:44.524601,\N,\N,Custom,80.0,0.0,\N
107237,171109,https://twitter.com/acesounderglass/status/173...,"Elizabeth Van Nostrand on X: ""Hat tip to @stua...",\N,\N,2023-12-14 17:07:44.530191,2023-12-14 17:07:44.530195,\N,\N,Custom,7.0,0.0,\N
107238,171110,https://twitter.com/JeremyLNeufeld/status/1734...,"(1) Jeremy Neufeld on X: ""The House Select Com...",\N,\N,2023-12-14 17:07:44.535702,2023-12-14 17:07:44.535707,\N,\N,Custom,100.0,0.0,\N
107239,171111,https://docs.google.com/document/d/1tMlNnvZRJU...,Report on 2023 Expert Survey on Progress in AI...,\N,\N,2023-12-14 17:07:44.541443,2023-12-14 17:07:44.541448,\N,\N,Custom,100.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107236,171108,https://twitter.com/aidan_mclau/status/1734621...,"(1) Aidan McLau on X: ""The irony of Mistral re...",\N,\N,2023-12-14 17:07:44.524596,2023-12-14 17:07:44.524601,\N,\N,Custom,80.0,0.0,\N
107237,171109,https://twitter.com/acesounderglass/status/173...,"Elizabeth Van Nostrand on X: ""Hat tip to @stua...",\N,\N,2023-12-14 17:07:44.530191,2023-12-14 17:07:44.530195,\N,\N,Custom,7.0,0.0,\N
107238,171110,https://twitter.com/JeremyLNeufeld/status/1734...,"(1) Jeremy Neufeld on X: ""The House Select Com...",\N,\N,2023-12-14 17:07:44.535702,2023-12-14 17:07:44.535707,\N,\N,Custom,100.0,0.0,\N
107239,171111,https://docs.google.com/document/d/1tMlNnvZRJU...,Report on 2023 Expert Survey on Progress in AI...,\N,\N,2023-12-14 17:07:44.541443,2023-12-14 17:07:44.541448,\N,\N,Custom,100.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107236,171108,https://twitter.com/aidan_mclau/status/1734621...,"(1) Aidan McLau on X: ""The irony of Mistral re...",\N,\N,2023-12-14 17:07:44.524596,2023-12-14 17:07:44.524601,\N,\N,Custom,80,0,\N
107237,171109,https://twitter.com/acesounderglass/status/173...,"Elizabeth Van Nostrand on X: ""Hat tip to @stua...",\N,\N,2023-12-14 17:07:44.530191,2023-12-14 17:07:44.530195,\N,\N,Custom,7,0,\N
107238,171110,https://twitter.com/JeremyLNeufeld/status/1734...,"(1) Jeremy Neufeld on X: ""The House Select Com...",\N,\N,2023-12-14 17:07:44.535702,2023-12-14 17:07:44.535707,\N,\N,Custom,100,0,\N
107239,171111,https://docs.google.com/document/d/1tMlNnvZRJU...,Report on 2023 Expert Survey on Progress in AI...,\N,\N,2023-12-14 17:07:44.541443,2023-12-14 17:07:44.541448,\N,\N,Custom,100,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)